In [2]:
'''
Import necessary packages
'''
import os
import pandas as pd
import numpy as np
from scipy.spatial.distance import cdist

import matplotlib.pyplot as plt
import imageio
from colormath.color_objects import sRGBColor, LabColor
from colormath.color_conversions import convert_color
from PIL import Image
# import cv2
from collections import Counter

np.set_printoptions(precision=3)

%matplotlib inline

In [3]:
def convert_to_lab(arr_rgb):
    '''
    This function will transform every rgb value in the list to lab. 
    '''
    
    list_lab = []
    for row in arr_rgb:
        r, g, b = row
        list_lab.append(convert_color(sRGBColor(r, g, b, is_upscaled=True), LabColor).get_value_tuple())
    return np.array(list_lab)


def convert_to_rgb(arr_lab):
    '''
    This function will transform every rgb value in the list to lab. 
    '''
    
    list_rgb = []
    for row in arr_lab:
        l, a, b = row
        list_rgb.append(convert_color(LabColor(l, a, b), sRGBColor).get_value_tuple())
    return np.array(list_rgb)

def find_the_cloeset_iscc(arr_img_lab, arr_iscc_lab):
    
    return arr_iscc_lab[np.argmin(cdist(arr_img_lab, arr_iscc_lab), 1)] 

# function to return key for any value 
def get_key(my_dict, val): 
    for key, value in my_dict.items(): 
         if val == value: 
                return key 
    return "key doesn't exist"
      

In [4]:
'''
Define the color card with ISCC_NBS
'''

# Read ISCC from local file
path_iscc = os.path.join('ISCC_NBS', 'ISCC_NBS_w3.csv')
df_iscc = pd.read_csv(path_iscc, names = ['color_name', 'hex'])

# Get the RGB and CIE-Lab values for each color in ISCC color card
df_iscc['RGB'] = ""
df_iscc['Lab'] = ""
for index, row in df_iscc.iterrows(): 
    df_iscc.loc[index, 'RGB'] = sRGBColor.new_from_rgb_hex(row['hex'])
    df_iscc.loc[index, 'Lab'] = convert_color(df_iscc.loc[index, 'RGB'], LabColor).get_value_tuple()

In [5]:
def img_to_vec(img_path):
    
    img = Image.open(path_img).convert('RGB')
    img = np.array(img)
    arr_img_lab = convert_to_lab(img.reshape(img.shape[0]*img.shape[1], 3))
    arr_iscc_lab = np.array(df_iscc['Lab'].tolist())
    quantized_img_lab = find_the_cloeset_iscc(arr_img_lab, arr_iscc_lab)
    quantized_img_rgb = convert_to_rgb(quantized_img_lab).reshape(img.shape[0], img.shape[1], 3)
    background_lab = np.array([95.7931405 , -0.15465805, -0.6054342])
    (unique, counts) = np.unique(quantized_img_lab, return_counts=True, axis=0)
    indices = np.argsort(counts)[::-1]
    dict_color_count = {}
    dict_color_name = {}
    sum_counts = sum(counts)


    for i in indices:
        if cdist(unique[i].reshape(1, 3), background_lab.reshape(1, 3)) < 0.001:
            continue
        index_iscc = np.where(cdist(df_iscc['Lab'].tolist(), unique[i].reshape(1, 3))<0.01)
        hex_value = df_iscc.loc[index_iscc[0][0], 'hex']
        color_name = df_iscc.loc[index_iscc[0][0], 'color_name']
        dict_color_count[hex_value] = counts[i]/sum_counts
        dict_color_name[hex_value] = color_name
    return dict_color_count, dict_color_name

In [6]:

path = os.path.join('images', 'blood')
df_vec = pd.DataFrame(columns = df_iscc['hex'].values) 
for img_name in os.listdir(path):
    if img_name.startswith('.'):
        continue
        
    path_img = os.path.join(path, img_name)
    dict_color, dict_temp_names = img_to_vec(path_img)
    print(dict_temp_names)
    for color_hex in df_vec.columns:
        if color_hex not in dict_color.keys():
            dict_color[color_hex] = 0
    df_vec = df_vec.append(dict_color,ignore_index=True,sort=False)
    print(df_vec)
    print(df_vec.size)

9;: &#39;Light_Grayish_Brown&#39;, &#39;#57554C&#39;: &#39;Olive_Gray&#39;, &#39;#3E322C&#39;: &#39;Dark_Grayish_Brown&#39;, &#39;#555555&#39;: &#39;Dark_Gray&#39;, &#39;#8DA399&#39;: &#39;Pale_Green&#39;, &#39;#635147&#39;: &#39;Grayish_Brown&#39;, &#39;#3B3121&#39;: &#39;Dark_Olive_Brown&#39;, &#39;#AE9B82&#39;: &#39;Light_Grayish_Yellowish_Brown&#39;, &#39;#483C32&#39;: &#39;Dark_Grayish_Yellowish_Brown&#39;, &#39;#BFB8A5&#39;: &#39;Yellowish_Gray&#39;, &#39;#5B504F&#39;: &#39;Brownish_Gray&#39;, &#39;#E8E3E5&#39;: &#39;Purplish_White&#39;, &#39;#8C8767&#39;: &#39;Light_Grayish_Olive&#39;, &#39;#25241D&#39;: &#39;Olive_Black&#39;, &#39;#BFB9BD&#39;: &#39;Light_Purplish_Gray&#39;, &#39;#593319&#39;: &#39;Deep_Brown&#39;, &#39;#C1B6B3&#39;: &#39;Pinkish_Gray&#39;, &#39;#C2AC99&#39;: &#39;Brownish_Pink&#39;, &#39;#E9E9ED&#39;: &#39;Bluish_White&#39;, &#39;#403D21&#39;: &#39;Dark_Olive&#39;, &#39;#B4BCC0&#39;: &#39;Light_Bluish_Gray&#39;, &#39;#3E1D1E&#39;: &#39;Dark_Reddish_Brown&#39;,

KeyboardInterrupt: 

In [13]:
def place_red_values_in_csv(dominant_color_names, dict_color_count, concern_color = 'Red'):
    dict_red = {}
    df_vec = pd.DataFrame(columns = df_iscc['hex'].values) 
    count = 0
    df_red_vec = pd.DataFrame(columns = df_iscc['hex'].values)
    for color_hex in dominant_color_names:
        name = dominant_color_names.get(color_hex)
        if concern_color in name:
            color_hex = get_key(dominant_color_names, name)
            count += 1
            dict_red[color_hex] = dict_color_count[color_hex]
            print(dict_red[color_hex])
            print(dict_color_count[color_hex])
            # count += dict_dominant_color_count[color_hex]
            # color_concerns.append(color_name)
            df_red_vec = df_red_vec.append(dict_red,ignore_index=True,sort=False)

    print(dict_red)
    print(count)
    return df_red_vec

In [14]:
path = os.path.join('images', 'blood')
df_red_vec = pd.DataFrame(columns = df_iscc['hex'].values) 
print(df_red_vec)
for img_name in os.listdir(path):
    if img_name.startswith('.'):
        continue
    path_img = os.path.join(path, img_name)
    dict_color, dict_names = img_to_vec(path_img)
    print('made it here')
    test_red_vec = place_red_values_in_csv(dict_names, dict_color, 'Red')
    # for color_hex in df_vec.columns:
    #     if color_hex not in dict_color.keys():
    #         dict_color[color_hex] = 0
    #delete break to do every photo 
    break
print(test_red_vec)

Empty DataFrame
Columns: [#FFB5BA, #EA9399, #E4717A, #F9CCCA, #DEA5A4, #C08081, #EAD8D7, #C4AEAD, #EAE3E1, #C1B6B3, #BE0032, #BC3F4A, #841B2D, #5C0923, #AB4E52, #722F37, #3F1728, #AD8884, #905D5D, #543D3F, #2E1D21, #8F817F, #5C504F, #282022, #FFB7A5, #F99379, #E66721, #F4C2C2, #D9A6A9, #C48379, #ECD5C5, #C7ADA3, #C2AC99, #E25822, #D9603B, #AA381E, #CB6D51, #9E4732, #B4745E, #882D17, #56070C, #A87C6D, #79443B, #3E1D1E, #977F73, #674C47, #43302E, #F38400, #FD943F, #ED872D, #BE6516, #FAB57F, #D99058, #AE6938, #80461B, #593319, #A67B5B, #6F4E37, #422518, #958070, #635147, #3E322C, #8E8279, #5B504F, #28201C, #F6A600, #FFC14F, #EAA221, #C98500, #FBC97F, #E3A857, #BE8A3D, #FAD6A5, #996515, #654522, #C19A6B, #826644, #4B3621, #AE9B82, #7E6D5A, #483C32, #F3C300, #FADA5E, #D4AF37, #AF8D13, #F8DE7E, #C9AE5D, #AB9144, #F3E5AB, #C2B280, #A18F60, #F0EAD6, #BFB8A5, #967117, #6C541E, #3B3121, #DCD300, #E9E450, #BEB72E, #9B9400, ...]
Index: []

[0 rows x 267 columns]
made it here
0.0019151138716356107


In [15]:
df_vec.shape

(6, 267)

In [16]:
test_red_vec.shape

(6, 267)

In [ ]:
df_vec.to_csv(os.path.join(path, 'blood.csv'), index=False)

In [17]:
test_red_vec.to_csv(os.path.join(path, 'test_onlyRed.csv'), index = False)